# Lab 1: Getting and Exploring Data with Minet and Python Pandas

What we will do:

1. Explain this programming environment
2. Scrape some Tweets based on a keyword search using the *minet* package
3. Use the pandas package to explore the data and generate some descriptive statistics and visualisations (unfortunately no networks today)
4. Learn some Python and command line principles on the way (if you didn't know it before)

There will be two versions of this so called Jupyter Notebook for you to follow along:

* One already filled out for you, in case you want to pay more attention on other things than typing or rather alter the code to try new things.
* Another one with the code 'cells' emptied for you to practice your Python typing skills alongside the lecturer (or maybe sometimes find even better solutions to the given problems)

Secret tip: If you want to try this at home, ChatGPT and Bing Chat got pretty good at generating code for you. However, you still should be able to make sure that the code they produced actually does what you want it to do. So you still have to learn some Python.

But now let's start.

## Get to know the minet package

Let's check whether minet is correctly setup in this programming environment.

You can always look up instructions on how to use it in its [documentation](https://github.com/medialab/minet/blob/master/docs/cli.md).

The output of this cell should be something like `minet 0.67.1`

In [ ]:
!minet --version

Let's call for help.

In [ ]:
!minet --help

We actually want twitter data, so let's try that

In [ ]:
!minet twitter

Not sure whether the API is still working, so we choose scraping.

In [ ]:
!minet twitter scrape -h

We're interested in discussions about Germany giving battle tanks to Ukraine. So, let's try to scrape 100 tweets, just to try our query, containing the word `Leopard` (the name of a German tank model most requested by Ukraine).

In [ ]:
!minet twitter scrape tweets -l 10 "Leopard"

Guess, we have to refine the query … 

In [ ]:
!minet twitter scrape tweets -l 10 "(ukraine Germany) AND (tank OR tanks OR leopard)"

Meh, still not good enough?

In [ ]:
!minet twitter scrape tweets -l 10 "(Ukraine Germany) AND (tank OR tanks OR leopard) AND (deliver OR delivery OR delivers)"

Ok, this looks better. But we want more tweets, and this will be too much to view here. So let's write to a CSV called `leo_tweets.csv`

In [ ]:
!minet twitter scrape tweets -l 10 "(Ukraine Germany) AND (tank OR tanks OR leopard) AND (deliver OR delivery OR delivers)" -o leo_tweets.csv

Now, open the CSV file on the left to have a look at it whether everything looks ok.

Then come back and we'll collect tweets since the beginning of this year.

(And go for a coffee in the meantime. Should take about 3 minutes.)

In [ ]:
!minet twitter scrape tweets "(Ukraine Germany) AND (tank OR tanks OR leopard) AND (deliver OR delivery OR delivers) since:2023-01-01" -o leo_tweets.csv

For the remainder of this tutorial we will use Pandas. Pandas is basically a swiss army knife for data wrangling and analysis in Python. Think of it as R, but in Python.

You can always look up its documentation [here](https://pandas.pydata.org/docs/user_guide/index.html).

First we need to import the package with `import pandas as pd`.

## Explore the Data

In [ ]:
import pandas as pd

Then we read in the data with `pd.read_csv`. You can always get help in Jupyter by writing a question mark behind a command and run the cell. Also, try using the (Shift+)TAB key for triggering autocompletion!

In [ ]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('leo_tweets.csv')

In [ ]:
# Display the DataFrame
df

Let's parse the dates with help of the documentation of the read_csv function.

In [ ]:
df = pd.read_csv('leo_tweets.csv', parse_dates=['local_time'])
df

First, let's see what columns are there

In [ ]:
df.columns

Which are the most retweeted tweets?

In [ ]:
# set display option for text column
pd.set_option('display.max_colwidth', 1000)
# sort by retweet count, and show top 10
df[['retweet_count','user_screen_name','local_time','text']].sort_values(ascending=False, by='retweet_count')[:10]

This is nice, but to get a birds-eye view of the data, we need to plot it.

In [ ]:
# Let's plot the retweet counts
df['retweet_count'].plot(kind='hist', bins=100, logy=True, xlabel='Retweet Count', ylabel='Frequency', title='Histogram of Retweet Counts')

In [ ]:
# Is there a relationship between retweet count and like count?
df.plot(kind='scatter', x='retweet_count', y='like_count', logx=True, logy=True, xlabel='Retweet Count', ylabel='Like Count', title='Scatterplot of Retweet Count vs. Like Count')

In [ ]:
# What about views?
df.plot(kind='scatter', x='retweet_count', y='impression_count', logx=True, logy=True, xlabel='Retweet Count', ylabel='View Count', title='Scatterplot of Retweet Count vs. View Count')

In [ ]:
# What about replies?
df.plot(kind='scatter', x='retweet_count', y='reply_count', logx=True, logy=True, xlabel='Retweet Count', ylabel='Reply Count', title='Scatterplot of Retweet Count vs. Reply Count')

In [ ]:
# And followers vs. retweets?
df.plot(kind='scatter', x='user_followers', y='retweet_count', logx=True, logy=True, xlabel='User Followers Count', ylabel='Retweet Count', title='Scatterplot of User Followers Count vs. Retweet Count')

In [ ]:
# Followers vs. views?
df.plot(kind='scatter', x='user_followers', y='impression_count', logx=True, logy=True, xlabel='User Followers Count', ylabel='View Count', title='Scatterplot of User Followers Count vs. View Count')

In [ ]:
# Finally, let's do a pie chart of the top 10 hashtags
df['hashtags'].value_counts()[:10].plot(kind='pie', title='Top 10 Hashtags')

In [ ]:
# This is not right, because the hashtags are not separated. Let's have a look at the data
df['hashtags'].value_counts()

In [ ]:
# Let's try to fix this
df['hashtags'].str.split('|', expand=True).stack().value_counts()[:10].plot(kind='pie', title='Top 10 Hashtags')

There's always more than one way to solve the same problem. Let's see who tweeted the most with the groupby and count command.

In [ ]:
df.groupby('user_screen_name')['id'].count().sort_values(ascending=False)

And let's make with the top 30 a nice bar plot with the plot function.

In [ ]:
df.groupby('user_screen_name')['id'].count().sort_values(ascending=False)[:30].plot(kind='bar')

Let's look at their user descriptions.

In [ ]:
# Group by screen name and count the number of tweets per user
top_30 = df.groupby('user_screen_name')['id'].count().sort_values(ascending=False)[:30]

# Merge the top 30 users with their descriptions
top_30_with_descriptions = pd.merge(top_30, df, left_index=True, right_on='user_screen_name')[['user_screen_name', 'user_description']]

# Drop duplicates
top_30_with_descriptions.drop_duplicates()

Let's look at the tweets of the most active account with 'boolean filtering'.

In [ ]:
# Identify the most active users by number of tweets
top_user = top_30_with_descriptions['user_screen_name'].iloc[0]

# Print the top user's screen name
print(top_user)

# Print the top user's tweets
df[df['user_screen_name'] == top_user][['local_time', 'text']].sort_values(by='local_time')

In [ ]:
# How many people did the top user reach?
df[df['user_screen_name'] == top_user]['impression_count'].sum()

In [ ]:
# Who has actually reached the most people?
df.groupby('user_screen_name')['impression_count', 'retweet_count', 'like_count'].sum().sort_values(by='impression_count', ascending=False)[:10]

Let's now look at tweets over time

In [ ]:
# Count the number of tweets per day
df.groupby(df["local_time"].dt.date)['id'].count().plot(kind="bar", figsize=(15,5), xlabel='Date', ylabel='Number of Tweets', title='Number of Tweets per Day')

There was a lot of activity on certain days. Let's look closer with 'boolean filtering'.

In [ ]:
# filter the dataframe to only include rows with local_time between 2023-01-25 and 2023-01-26, sort by time, and show only the date and text columns
df[(df['local_time'] > '2023-01-25') & (df['local_time'] < '2023-01-26')].sort_values(by='local_time')[['local_time', 'text']]

And now, to have some kind of network analysis at least, let's look at who got the most mentions

In [ ]:
# 1. create a dataframe from a column containing a string of names separated by |
mentioned_names = df[['mentioned_names']]

# 2. expand the string of names into a list of names
expanded = mentioned_names['mentioned_names'].str.split('|', expand=True)

# 3. print the expanded list of names
expanded

In [ ]:
# 4. stack the list of names into a single column
stacked = expanded.stack()

stacked

In [ ]:
# 5. count the number of times each name appears
most_mentioned = stacked.value_counts()

In [ ]:
most_mentioned[100::-1].plot(kind='barh', figsize=(5,15))

# Thanks for your attention! Any Questions?

Ask now or @flxvctr(@mas.to) on Twitter or Mastodon.

That's it. For homework: 

1. Try to generate a DataFrame with one column containing the users tweeting and the other column containing the users mentioned in the tweets.
2. Export the DataFrame to a CSV file.
3. Download and install Gephi and import the CSV file to create a network graph.

Don't worry if you do not succeed. We will do this together in the next lab.